<a href="https://colab.research.google.com/github/JuanCamiloGB/Signals-and-Systems/blob/master/Taller02/3.%20Transformada%20de%20Fourier/4_4_Rectificador_RC_THD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Aplicación en circuitos eléctricos
#### Consulte en que consiste la distorsión total de armónicos (Total Harmonic Distortion-(THD)) y el factor de potencia en un circuito eléctrico. ¿Como puede calcularse el THD desde la FFT? ¿Como puede calcularse el factor de potencia con base al THD? Genere un ejemplo ilustrativo en su software de preferencia para el cálculo del THD y el factor de potencia para un rectificador de onda completa con carga: i) netamente resistiva y ii) carga RC en serie. Establezca las condiciones necesarias para las simulaciones y pruebe con diferentes valores de R y C. Discuta los resultados obtenidos

###La Distorsión Armónica Total (THD): 
Es una medida de cuánto se «distorsiona» o se cambia la forma de onda del voltaje o de la corriente de su forma de onda sinusoidal convencional. La energía proviene de su suministrador eléctrico en forma de onda sinusoidal limpia. A medida que atraviesa diferentes tipos de cargas, el voltaje y la corriente se utilizan a diferentes tasas, lo que provoca que la distorsión se refleje desde la carga hacia el sistema.
La distorsión ocurre principalmente en múltiplos de la frecuencia de la portadora (50 o 60 Hz) que se conocen como armónicos. Por ejemplo, el 3er armónico en una línea de 60 Hz sería de 180 Hz, el 7 ° sería 420 Hz. La THD es el porcentaje acumulado de distorsión para todos los tipos de armónicos en relación a la potencia total. La distorsión se mide por separado para la corriente (THDI) y el voltaje (THDV).

###Factor de Potencia: 

Los receptores eléctricos transforman la energía eléctrica en otro tipo de energía, por ejemplo en energía luminosa (una lámpara), calorífica (un radiador o una plancha), motriz o mecánica (un motor) o sonora (un timbre). El problema es que en estos receptores toda la energía eléctrica que consumen, la mayoría de las veces, no se transforma por completo en energía útil (luz, calor, etc..
 
El factor de potencia es una medida de la eficiencia o rendimiento eléctrico de un receptor o sistema eléctrico, En términos simples, este indicador mide el aprovechamiento de la energia, a la hora de convertirlo en potencia útil, como luz, calor o movimiento mecánico.

En términos técnicos, es la ratio de potencia activa o útil medida en kilovatios (kW) respecto a la potencia aparente total (potencia activa y reactiva) medida en kilovoltio-amperios.

$Factor \ de \ Potencia = \frac{Potencia \ activa \ (P'[W]')}{Potencia \ aparente \ (S'[VA]')}$

NOTA: a mayor potencia activa y menor potencia reactiva tendremos un factor de potencia más cercano a 1 (el ideal).

In [ ]:
#paquetes de para simulación
import numpy as np
import scipy
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.signal as sig
import scipy.optimize as opt
from IPython.display import Image
import sympy as sym 
sym.init_session() 

# i) netamente resistivo 

In [ ]:
s = sym.symbols('s', complex=True)
t, R, L, C = sym.symbols('t R L C', positive=True)
X = sym.Function('X')(s)

Y = 1/(R*C*s + 1) * X
Y

In [ ]:
R_v = 1000 #valores de simulacion de R
C_v =  0 # valores de simulacion de C
Y_RC = Y.subs(R, R_v).subs(C, C_v)
Y_RC

In [ ]:
num = np.array([1]) #coeficientes del numerador de la funcion de transferencia como numpy array
den = np.array([R_v*C_v,1])#coeficientes del denominador de la funcion de transferencia como numpy array
G_n = sig.TransferFunction(num, den) #funcion de transferencia en
print(num)
print(den)

In [ ]:
Fo = 60 #frec alimentación
Fs = 30*Fo #frecuencia muestreo
To = 1/Fo #periodo fundamental
Ts = 1/Fs #periodo muestreo
t = np.arange(0, 5*To,Ts) # se simulan 5 peridos de alimentación con un muestreo segun Ts
A = 120 #amplitud entrada
#función de entrada aproximada como onda senoidal rectificada completa
in_o = A*(np.sin(2*np.pi*Fo*t))
rec_c = sig.square(2*np.pi*Fo*t)#función tren de pulsos para simular señal rectificada 
#Para rectificador de media onda: 
rec_m = 0.5*(sig.square(2*numpy.pi*Fo*t)+1)
in_ = in_o * rec_c #si se quiere probar con rec media onda multiplicar por rec_m
out = G_n.output(in_, T=t)[1] #evaluar salida ante entrada rectificada mediante aproximacion numérica de scipy

In [ ]:
#señales de alimentación y tren de pulsos
plt.plot(t,in_o,label='Alimentación')
plt.legend()
plt.show()

In [ ]:
plt.plot(t,rec_c,label='Tren Pulsos')
plt.legend()
plt.show()

In [ ]:
plt.plot(t, in_,label='In(t)')#entrada rectificada
plt.plot(t, out,label='Out(t)')#salida del circuito
plt.xlabel('t [s]')
plt.ylabel('out(t)')
plt.show()

In [ ]:
vfre = np.fft.rfftfreq(len(out),1/Fs) #vector de frecuencia en Hz segun tamaño del vector y tiempo de muestreo
Xf = np.fft.rfft(out)

plt.plot(vfre,abs(Xf/len(out)))
plt.xlabel('Frecuencia [Hz]')
plt.ylabel('$|X(f)|$')
plt.show()

  ### Conclusiones: 
  Como es un circuito puramente resistivo, al no tener elementos reactivos, la corriente y la tension se encuentran en fase $\theta = 0$, por lo tanto el factor de potencia es 1.

# ii) Carga RC en serie

In [ ]:
s = sym.symbols('s', complex=True)
t, R, L, C = sym.symbols('t R L C', positive=True)
X = sym.Function('X')(s)

Y = 1/(R*C*s + 1) * X
Y

In [ ]:
R_v = 1000 #valores de simulacion de R
C_v =  1e-6 # valores de simulacion de C
Y_RC = Y.subs(R, R_v).subs(C, C_v)
Y_RC

In [ ]:
num = np.array([1]) #coeficientes del numerador de la funcion de transferencia como numpy array
den = np.array([R_v*C_v,1])#coeficientes del denominador de la funcion de transferencia como numpy array
G_n = sig.TransferFunction(num, den) #funcion de transferencia en
print(num)
print(den)

In [ ]:
Fo = 60 #frec alimentación
Fs = 30*Fo #frecuencia muestreo
To = 1/Fo #periodo fundamental
Ts = 1/Fs #periodo muestreo
t = np.arange(0, 5*To,Ts) # se simulan 5 peridos de alimentación con un muestreo segun Ts
A = 120 #amplitud entrada
#función de entrada aproximada como onda senoidal rectificada completa
in_o = A*(np.sin(2*np.pi*Fo*t))
rec_c = sig.square(2*np.pi*Fo*t)#función tren de pulsos para simular señal rectificada 
#Para rectificador de media onda: 
rec_m = 0.5*(sig.square(2*numpy.pi*Fo*t)+1)
in_ = in_o * rec_c #si se quiere probar con rec media onda multiplicar por rec_m
out = G_n.output(in_, T=t)[1] #evaluar salida ante entrada rectificada mediante aproximacion numérica de scipy

In [ ]:
#señales de alimentación y tren de pulsos
plt.plot(t,in_o,label='Alimentación')
plt.legend()
plt.show()

In [ ]:
plt.plot(t,rec_c,label='Tren Pulsos')
plt.legend()
plt.show()

In [ ]:
plt.plot(t, in_,label='In(t)')#entrada rectificada
plt.plot(t, out,label='Out(t)')#salida del circuito
plt.xlabel('t [s]')
plt.ylabel('out(t)')
plt.show()

In [ ]:
vfre = np.fft.rfftfreq(len(out),1/Fs) #vector de frecuencia en Hz segun tamaño del vector y tiempo de muestreo
Xf = np.fft.rfft(out)

plt.plot(vfre,abs(Xf/len(out)))
plt.xlabel('Frecuencia [Hz]')
plt.ylabel('$|X(f)|$')
plt.show()